# Lyme disease detection

In [49]:
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D, Cropping2D, concatenate, BatchNormalization, GlobalAveragePooling2D, Conv2DTranspose
from keras.layers import Activation, Dropout, Flatten, Dense, Input
#from tensorflow.python.keras import applications
#model = applications.VGG16(weights='imagenet', include_top=False)

config = tf.ConfigProto( device_count = {'GPU': 8 , 'CPU': 32} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5526038941983446001
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15589952717
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12774229198510523427
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:0a:00.0, compute capability: 7.0"
]


## Creating a Convolutional Neural Network

In [50]:
from keras.applications.xception import Xception, preprocess_input
from keras.models import Model

base_model = Xception(input_shape=(600, 600, 3), weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax', name='prediction')(x)
model = Model(base_model.input, predictions)


## Compiling

In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Creating Image Generators
Based on ImageDataGenerator class.

In [58]:
# Каталог с данными для обучения
train_dir = 'processed_dataset/train'
# Каталог с данными для проверки
val_dir = 'processed_dataset/val'
# Каталог с данными для тестирования
test_dir = 'processed_dataset/test'
# Размеры изображения
img_width, img_height = 600, 600
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 40
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 442
# Количество изображений для проверки
nb_validation_samples = 23
# Количество изображений для тестирования
nb_test_samples = 23

In [59]:
datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


Generator for training

In [60]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 442 images belonging to 2 classes.


Generator for checking and adjusting the hyperparameters

In [61]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 23 images belonging to 2 classes.


Generator for testing

In [62]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 23 images belonging to 2 classes.


# Learning the model using generators

train_generator - for learning

validation_data - for checking and adjusting

In [63]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/40
55/55 [==============================] - 131s 2s/step - loss: 0.6838 - acc: 0.7318 - val_loss: 0.7251 - val_acc: 0.3750
Epoch 2/40
55/55 [==============================] - 124s 2s/step - loss: 0.6538 - acc: 0.7455 - val_loss: 10.0738 - val_acc: 0.3750
Epoch 3/40
55/55 [==============================] - 125s 2s/step - loss: 0.5743 - acc: 0.7568 - val_loss: 4.8641 - val_acc: 0.3750
Epoch 4/40
55/55 [==============================] - 123s 2s/step - loss: 0.5834 - acc: 0.7568 - val_loss: 4.0284 - val_acc: 0.3750
Epoch 5/40
55/55 [==============================] - 127s 2s/step - loss: 0.5666 - acc: 0.7681 - val_loss: 2.8531 - val_acc: 0.3750
Epoch 6/40
55/55 [==============================] - 122s 2s/step - loss: 0.5414 - acc: 0.7499 - val_loss: 1.9788 - val_acc: 0.3750
Epoch 7/40
55/55 [==============================] - 125s 2s/step - loss: 0.6076 - acc: 0.7408 - val_loss: 1.0586 - val_acc: 0.3750
Epoch 8/40
55/55 [==============================] - 123s 2s/step - loss: 0.5564 - 

# Evaluating the accuracy score of the system

In [65]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Accuracy on the test set: %.2f%%" % (scores[1]*100))

Accuracy on the test set: 81.25%


## Print the name of the output node

In [63]:
print(model.outputs)

[<tf.Tensor 'prediction/Softmax:0' shape=(?, 2) dtype=float32>]


## Feed a random Tensor to make sure the system is working

In [45]:
import numpy as np
model.predict(np.random.rand(1, 600, 600, 3))

array([[0.96820104, 0.03179901]], dtype=float32)

## Code to predict on a single image

In [81]:
from tensorflow.python.keras.preprocessing import image

x = image.load_img('processed_dataset/test/EM/bull_16.jpg', target_size=(600,600))
x = image.img_to_array(x)
x = x.reshape((1,) + x.shape)
x = x/255.
model.predict(x)

array([[0.9974831 , 0.00251691]], dtype=float32)

## Save the model

In [82]:
model.save("model_600x600_83perc.h5")